In [11]:
PROJECT="vertex-ai-by-example"
COMPONENT_NAME="bash-io"
BUCKET_NAME=f"gs://{PROJECT}"
print(BUCKET_NAME)

gs://vertex-ai-by-example


In [25]:
!gcloud config set project $PROJECT
!gcloud services enable notebooks.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable aiplatform.googleapis.com
!echo "Note: it can take a while for service accounts to get created with correct permissions"


Updated property [core/project].
Operation "operations/acf.p2-662478959901-9f44e5a9-2d34-4266-bbd3-42574aaae7f6" finished successfully.
Note: it can take a while for service accounts to get created with correct permissions


In [17]:
!gsutil mb $BUCKET_NAME

Creating gs://vertex-ai-by-example/...


In [18]:
!gcloud builds submit --tag gcr.io/$PROJECT/$COMPONENT_NAME .

Creating temporary tarball archive of 8 file(s) totalling 107.7 KiB before compression.
Uploading tarball of [.] to [gs://vertex-ai-by-example_cloudbuild/source/1636932805.240563-4ab676bb014b4e219e40ce38dcb74f0c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/vertex-ai-by-example/locations/global/builds/8c288d41-9553-438e-8e15-88cc3d140a66].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/8c288d41-9553-438e-8e15-88cc3d140a66?project=662478959901].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "8c288d41-9553-438e-8e15-88cc3d140a66"

FETCHSOURCE
Fetching storage object: gs://vertex-ai-by-example_cloudbuild/source/1636932805.240563-4ab676bb014b4e219e40ce38dcb74f0c.tgz#1636932805536699
Copying gs://vertex-ai-by-example_cloudbuild/source/1636932805.240563-4ab676bb014b4e219e40ce38dcb74f0c.tgz#1636932805536699...
/ [1 files][ 17.0 KiB/ 17.0 KiB]                                                
Operation co

In [17]:
!pip install kfp --upgrade

In [19]:
from kfp.v2 import dsl, components, compiler
from kfp import components

bash_io_step = components.load_component_from_file("./component.yaml")

@dsl.pipeline(
    name="bash-io-pipeline", 
    pipeline_root=f"{BUCKET_NAME}/{COMPONENT_NAME}",
    enable_caching=False,
)
def bash_io_pipeline(project_id: str):
    bash_io_step(input_1="input1value", parameter_1=4)

jobspec_filename = f"{COMPONENT_NAME}-jobspec.json"

compiler.Compiler().compile(
    pipeline_func=bash_io_pipeline, 
    package_path=jobspec_filename
)
print(jobspec_filename)


TypeError: pipeline() got an unexpected keyword argument 'enable_caching'

In [13]:
!gsutil cp $jobspec_filename $BUCKET_NAME/jobspecs/$jobspec_filename

Copying file://bash-io-jobspec.json [Content-Type=application/json]...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      


In [20]:
import kfp

client = kfp.Client() # if this errors then pipelines hasn't been enabled

Failed to load kube config.


MaxRetryError: HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: /apis/v1beta1/healthz (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6e74fa9910>: Failed to establish a new connection: [Errno 111] Connection refused'))